## Uppsalakyrkogård kulturpersoner
version 0.1
* [This Notebook](https://github.com/salgo60/open-data-examples/blob/master/Uppsalakyrkogård%20kulturpersoner.ipynb)
* [Map](https://kulturpersoner.uppsalakyrkogardar.se/karta/) / [Lista](https://kulturpersoner.uppsalakyrkogardar.se/galleri/)
* [GIST](https://gist.github.com/salgo60/34e9a6bfc223687951ad827bf84b330d)
* [SPARQL](https://w.wiki/XkL)


#### Other sources we sync
* [Kulturpersoner Uppsalakyrkogård](https://github.com/salgo60/open-data-examples/blob/master/Uppsalakyrkogård%20kulturpersoner.ipynb)
* [Litteraturbanken](https://github.com/salgo60/open-data-examples/blob/master/Litteraturbanken%20Author.ipynb)
* [Nobelprize.org](https://github.com/salgo60/open-data-examples/blob/master/Nobel%20API.ipynb)
* [SBL](https://github.com/salgo60/open-data-examples/blob/master/SBL.ipynb)
* [SKBL](https://github.com/salgo60/open-data-examples/blob/master/Svenskt%20Kvinnobiografiskt%20lexikon%20part%203.ipynb)  
* [Svenska Akademien](https://github.com/salgo60/open-data-examples/blob/master/Svenska%20Akademien.ipynb)  


In [1]:
from datetime import datetime
start_time = datetime.now()
print("Last run: ", start_time)

Last run:  2022-12-31 09:43:43.495118


In [2]:
#https://kulturpersoner.uppsalakyrkogardar.se/galleri/
import requests
import urllib.request
import pandas as pd
import time
from bs4 import BeautifulSoup
def cleanupstring(s):
    out_s = s
    while '  ' in out_s:
        out_s = out_s.strip().replace('  ', ' ')
    return out_s, len(s)-len(out_s)

dfUppsala = pd.DataFrame(columns=["url","name"])

urls = [
    'https://kulturpersoner.uppsalakyrkogardar.se/galleri/']
for url in urls:
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")
    soup.findAll('a')
    one_a_tag = soup.findAll('a')[0]
    time.sleep(1)
    for i in range(1, len(soup.findAll('a'))):
        one_a_tag = soup.findAll('a')[i]
        one_a_tag_href = one_a_tag['href']
        if   "karta" not in one_a_tag_href \
                and not "Search" in one_a_tag.text \
                and not "Start" in one_a_tag.text \
                and not "Vandringar" in one_a_tag.text \
                and not "Guidad" in one_a_tag.text \
                and not "galleri" in one_a_tag_href \
                and not "om-oss"  in one_a_tag_href \
                and not "vandringar"  in one_a_tag_href \
                and not "kyrkogardsexpedition"  in one_a_tag_href:
            name = one_a_tag.text.replace("https://kulturpersoner.uppsalakyrkogardar.se/","").replace("-"," ").replace("%C3%84","Ä")
            #print(name,  "-", type(one_a_tag_href),one_a_tag_href)
            #dfUppsala = dfUppsalaList.append({'name': name,'url':one_a_tag_href})
            dfUppsala = pd.concat([dfUppsala, pd.DataFrame.from_records([{ 'name': name, 'url':one_a_tag_href }])], ignore_index=True)
  


In [3]:
dfUppsala.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 131 entries, 0 to 130
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   url     131 non-null    object
 1   name    131 non-null    object
dtypes: object(2)
memory usage: 2.2+ KB


In [4]:
"Uppsalakyrkogård_kulturpersoner.csv"

'Uppsalakyrkogård_kulturpersoner.csv'

In [5]:
import json,sys
import pandas as pd 
from SPARQLWrapper import SPARQLWrapper, JSON
endpoint_url = "https://query.wikidata.org/"

def get_sparql_dataframe(endpoint_url, query):
    """
    Helper function to convert SPARQL results into a Pandas data frame.
    """
    user_agent = "salgo60/%s.%s" % (sys.version_info[0], sys.version_info[1])
 
    sparql = SPARQLWrapper(endpoint_url, agent=user_agent)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    result = sparql.query()

    processed_results = json.load(result.response)
    cols = processed_results['head']['vars']

    out = []
    for row in processed_results['results']['bindings']:
        item = []
        for c in cols:
            item.append(row.get(c, {}).get('value'))
        out.append(item)

    return pd.DataFrame(out, columns=cols)

In [6]:
end = datetime.now()
print("Ended: ", end) 
print('Time elapsed (hh:mm:ss.ms) {}'.format(datetime.now() - start_time))

Ended:  2022-12-31 09:43:45.370485
Time elapsed (hh:mm:ss.ms) 0:00:01.875605
